In [ ]:
from sklearn.model_selection import StratifiedKFold
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
import numpy as np
import tensorflow as tf
from models import cnn_models as cnnmodel
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("GPU(s) disponibles:", len(gpus))
else:
    print("No se encontraron GPUs disponibles.")

def normResults(y_pred):
    yp = []
    for row in y_pred:
        valmax = max(row)
        for i in range(len(row)):
            if row[i] == valmax:
                row[i] = 1
            else:
                row[i] = 0
        
        yp.append(row)
    
    return yp

def environment_model_graph(data1, name1, data2, name2, data3, name3, data4, name4, rootsave, nameSave = "graph", var_title="graph", xlabel="Axes X", ylabel="Axes Y"):
    num_epochs = len(data1[0])
    x = [i for i in range(num_epochs)]
    
    y1 = []
    y2 = []
    y3 = []
    y4 = []
    for i,j,m,n in zip(data1[0], data2[0], data3[0], data4[0]):
        y1.append(i)
        y2.append(j)
        y3.append(m)
        y4.append(n)
            
    sns.set(style="darkgrid")
    
    fig, axes = plt.subplots(2,1,figsize=(10,8))
    
    axes[0].plot(x, y1, label=name1)
    axes[0].plot(x, y2, label=name2)
    axes[0].set_title(var_title)
    axes[0].set_xlabel(xlabel)
    axes[0].set_ylabel(ylabel)
    axes[0].legend()
    
    axes[1].plot(x, y3, label=name3)
    axes[1].plot(x, y4, label=name4)
    axes[1].set_xlabel(xlabel)
    axes[1].set_ylabel(ylabel)
    axes[1].legend()
    
    plt.tight_layout()
    
    name = nameSave+'.png'
    plt.savefig(rootsave+name, dpi=300)

def add_column_df(df, kfold,precision, recall, f1_list, weighted_avg):
    
    df['kfold'] = [kfold, kfold]
    df['class'] = [0,1]
    df['precision'] = precision
    df['recall'] = recall
    df['f1'] = f1_list
    df['weighted_avg'] = [weighted_avg, weighted_avg]
    
    return df



"""Paths and data"""
root = '../experimental_test2/'
root_graphs = root+'figures/'
#load data
data = np.load("../../../datasets/TCIA_SegPC_dataset/crops/dataset.npy")
labels = np.load("../../../datasets/TCIA_SegPC_dataset/crops/labels.npy")

data = data/255.0


""" Train and test """
skfolds = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
columns = ['model', 'kfold', 'class', 'accuracy', 'precision', 'recall', 'f1', 'weighted_avg']
df = pd.DataFrame(columns=columns)
list_models = [cnnmodel.model_3(), cnnmodel.model_4(), cnnmodel.model_6(), cnnmodel.model_7(), cnnmodel.model_8()]

             
for nomodel in range(len(list_models)):
    num_fold = 0 
    model = list_models[nomodel]
    columns = ['kfold', 'phase', 'precision', 'recall', 'f1', 'weighted_avg']
    df = pd.DataFrame(columns=columns)
    df_parcial = pd.DataFrame(columns=columns)
    for train_index, test_index in skfolds.split(data, labels):
                
        #divide data
        X_train_folds, X_test_folds = data[train_index], data[test_index]
        y_train_folds, y_test_folds = labels[train_index], labels[test_index]

            
        
        y_train_folds = to_categorical(y_train_folds)
        y_test_folds = to_categorical(y_test_folds)
        
        """ train"""
        #define callback EarlyStopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=30, verbose=1, restore_best_weights=True)
        #fit model
        history = model.fit(X_train_folds, y_train_folds, validation_data = (X_test_folds, y_test_folds), epochs=500, batch_size=64, callbacks=[early_stopping])

        accuracy_model=[]
        accuracy_model.append(model.evaluate(X_test_folds, y_test_folds))
        y_pred = model.predict(X_test_folds)          

        yp = [] 
        yp = normResults(y_pred)

        results = []
        n_correct = sum(yp == y_test_folds)
        results.append(n_correct/len(y_pred))
        
        
        yp2 = np.array(yp)
        precision = []
        recall = []
        f1_list = []
        for i in range(2):
            precision.append(precision_score(y_test_folds[:,i],yp2[:,i]))
            recall.append(recall_score(y_test_folds[:,i],yp2[:,i]))
            f1 = f1_score(y_test_folds[:,i],yp2[:,i])
            f1_list.append(f1)


        weighted_avg = []
        f1_list = np.array(f1_list)    
        weight = np.array(list(sum(y_test_folds == 1)))
        weighted_sum = np.sum(f1_list * weight)
        total_weight = np.sum(weight)
        weighted_avg.append(weighted_sum / total_weight)
        
        #df = add_column_df(df,nomodel, num_fold, accuracy_model, precision, recall, f1_list, weighted_sum / total_weight)

        num_fold += 1
        df_parcial = add_column_df(df_parcial,num_fold, precision, recall, f1_list, weighted_sum / total_weight)
        df = pd.concat([df, df_parcial])

    root_df = root+"sheets/"
    name_excel = "result_model_" + str(nomodel) + ".xlsx"
    df.to_excel(root_df + name_excel, index=False)

No se encontraron GPUs disponibles.


e:\python\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - accuracy: 0.6264 - loss: 0.6740 - val_accuracy: 0.6597 - val_loss: 0.6666
Epoch 2/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.6445 - loss: 0.6557 - val_accuracy: 0.6597 - val_loss: 0.6249
Epoch 3/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.6503 - loss: 0.6361 - val_accuracy: 0.6597 - val_loss: 0.4915
Epoch 4/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.6521 - loss: 0.5121 - val_accuracy: 0.8202 - val_loss: 0.4902
Epoch 5/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.8328 - loss: 0.4700 - val_accuracy: 0.8467 - val_loss: 0.4299
Epoch 6/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.8472 - loss: 0.4455 - val_accuracy: 0.8499 - val_loss: 0.4196
Epoch 7/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.8357 - loss: 0.4196 - val_accuracy: 0.8395 - val_loss: 0.4164
Epoch 8/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.8555 - loss: 0.3909 - val_accuracy: 0.

<ipython-input-1-759a699430bc>:159: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_parcial])


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.8881 - loss: 0.2714 - val_accuracy: 0.9510 - val_loss: 0.1543
Epoch 2/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9131 - loss: 0.2479 - val_accuracy: 0.9655 - val_loss: 0.1062
Epoch 3/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9183 - loss: 0.1896 - val_accuracy: 0.9631 - val_loss: 0.1191
Epoch 4/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9216 - loss: 0.1867 - val_accuracy: 0.9574 - val_loss: 0.1517
Epoch 5/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.9539 - loss: 0.1359 - val_accuracy: 0.9622 - val_loss: 0.1513
Epoch 6/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9271 - loss: 0.1740 - val_accuracy: 0.9414 - val_loss: 0.1413
Epoch 7/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.9324 - loss: 0.1607 - val_accuracy: 0.9526 - val_loss: 0.1227
Epoch 8/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9411 - loss: 0.1470 - val_accuracy: 0.9518 - val_l

...

20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.9151 - loss: 0.2680 - val_accuracy: 0.9574 - val_loss: 0.1861
Epoch 2/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9067 - loss: 0.2282 - val_accuracy: 0.9703 - val_loss: 0.1066
Epoch 3/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.9262 - loss: 0.1717 - val_accuracy: 0.9574 - val_loss: 0.1420
Epoch 4/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9257 - loss: 0.1735 - val_accuracy: 0.9719 - val_loss: 0.1002
Epoch 5/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9507 - loss: 0.1390 - val_accuracy: 0.9582 - val_loss: 0.1153
Epoch 6/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9253 - loss: 0.1866 - val_accuracy: 0.9639 - val_loss: 0.1223
Epoch 7/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9328 - loss: 0.1510 - val_accuracy: 0.9639 - val_loss: 0.1072
Epoch 8/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9513 - loss: 0.1175 - val_accuracy: 0.9679 - val_l

<ipython-input-1-759a699430bc>:159: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_parcial])


20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 523ms/step - accuracy: 0.8842 - loss: 0.2861 - val_accuracy: 0.9550 - val_loss: 0.1482
Epoch 2/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 583ms/step - accuracy: 0.8924 - loss: 0.2898 - val_accuracy: 0.9398 - val_loss: 0.1443
Epoch 3/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 457ms/step - accuracy: 0.9104 - loss: 0.2172 - val_accuracy: 0.9582 - val_loss: 0.1303
Epoch 4/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 481ms/step - accuracy: 0.9010 - loss: 0.2291 - val_accuracy: 0.9542 - val_loss: 0.1305
Epoch 5/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 457ms/step - accuracy: 0.9158 - loss: 0.1840 - val_accuracy: 0.9486 - val_loss: 0.1313
Epoch 6/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 482ms/step - accuracy: 0.9241 - loss: 0.1908 - val_accuracy: 0.9470 - val_loss: 0.1515
Epoch 7/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 453ms/step - accuracy: 0.9350 - loss: 0.1685 - val_accuracy: 0.9582 - val_loss: 0.1099
Epoch 8/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 450ms/step - accuracy: 0.9329 - loss: 0.1593 - val_accuracy: 0.

<ipython-input-1-759a699430bc>:159: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_parcial])


20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 486ms/step - accuracy: 0.8924 - loss: 0.2880 - val_accuracy: 0.9510 - val_loss: 0.1767
Epoch 2/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 481ms/step - accuracy: 0.8942 - loss: 0.2498 - val_accuracy: 0.9534 - val_loss: 0.1427
Epoch 3/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 445ms/step - accuracy: 0.8921 - loss: 0.2434 - val_accuracy: 0.9438 - val_loss: 0.1703
Epoch 4/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 442ms/step - accuracy: 0.9065 - loss: 0.2344 - val_accuracy: 0.9422 - val_loss: 0.1407
Epoch 5/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 445ms/step - accuracy: 0.8894 - loss: 0.2727 - val_accuracy: 0.9285 - val_loss: 0.1664
Epoch 6/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 482ms/step - accuracy: 0.9136 - loss: 0.2042 - val_accuracy: 0.9357 - val_loss: 0.1469
Epoch 7/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 443ms/step - accuracy: 0.9336 - loss: 0.1815 - val_accuracy: 0.9446 - val_loss: 0.1460
Epoch 8/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 452ms/step - accuracy: 0.9316 - loss: 0.1547 - val_accuracy: 0.9